In [1]:
import cobra
from cobra import Model, Reaction, Metabolite
import pandas as pd
import numpy as np

# Import Biopython modules to interact with KEGG
from Bio import SeqIO
from Bio.KEGG import REST
from Bio.KEGG.KGML import KGML_parser

import re

In [92]:
df = pd.read_excel('KEGG_Reactions.xlsx')
print(len(df))
df.head()

1497


,EC number,KEGG reaction ID,Reaction_Ids,Reaction_metNames,Newman,TCH1516,USA300,Mu3,JH9,JH1,COL,RF122,Mu50,MW2,MSSA476,MRSA252,N315
0,1.1.1.100,R02767,C01271 + C00006 <=> C00685 + C00005 + C00080,(3R)-3-Hydroxyacyl-[acyl-carrier protein] + NA...,+,+,+,+,+,+,+,+,+,+,+,+,+
1,1.1.1.100,R04533,C04618 + C00006 <=> C05744 + C00005,(3R)-3-Hydroxybutanoyl-[acyl-carrier protein] ...,+,+,+,+,+,+,+,+,+,+,+,+,+
2,1.1.1.100,R04534,C04619 + C00006 <=> C05753 + C00005,(3R)-3-Hydroxydecanoyl-[acyl-carrier protein] ...,+,+,+,+,+,+,+,+,+,+,+,+,+
3,1.1.1.100,R04536,C04620 + C00006 <=> C05750 + C00005,(3R)-3-Hydroxyoctanoyl-[acyl-carrier protein] ...,+,+,+,+,+,+,+,+,+,+,+,+,+
4,1.1.1.100,R04543,C04633 + C00006 <=> C05762 + C00005,(3R)-3-Hydroxypalmitoyl-[acyl-carrier protein]...,+,+,+,+,+,+,+,+,+,+,+,+,+


### Reaction statistics

In [3]:
#print(list(df.keys())[4:])
total_reaction_number = 0
num_reactions = []

for item in list(df.keys())[4:]:
    series = df[item].value_counts()
    for f,g in series.iteritems():
        if f == '+':
            total_reaction_number+= g
            num_reactions.append(g)
mean_reaction_number = total_reaction_number/13
print(mean_reaction_number)
print(len(num_reactions))
print('Mean: ', np.mean(num_reactions))
print('Std: ', np.std(num_reactions))

1481.3076923076924
13
Mean:  1481.3076923076924
Std:  14.47217284937713


### Gather reaction information
- Given:
    - ID
    - Lower Bound (based on equation)
    - Upper Bound (based on equation)
- Needed:
    - Reaction Name 

In [4]:
#initialize List for reaction names
react_name_dict = {}

#get KEGG reaction

#result =REST.kegg_find('reaction', 'R02767').read()

for react in df['KEGG reaction ID'].tolist():
    try:
        result = REST.kegg_get(react).read()

        for line in result.split('\n'):
            if 'NAME' in line:
                react_name_dict[react]=line[12:]
    except:
        #print(react)
        react_name_dict[react] = react

        
print(len(react_name_dict))

1297


### Gather metabolite information:
- Given:
    - ID
- Needed:
    - Name
    - Compartment
    - Chemical formula
    (- Charge)

In [69]:
ListOfMetabolites = []

for reaction in df['Reaction_Ids'].tolist():
    splitting = reaction.split()
    for split in splitting:
        if '(?' in split or '(N' in split: 
            if split not in ListOfMetabolites:
                if '[e]' in split and split[:-3] not in ListOfMetabolites:
                    ListOfMetabolites.append(split.strip()[:-3])
                elif '[e]' not in split:
                    ListOfMetabolites.append(split.strip())
        elif '+' not in split and '>' not in split and '<' not in split and '(' not in split:
            if split not in ListOfMetabolites:
                if '[e]' in split and split[:-3] not in ListOfMetabolites:
                    ListOfMetabolites.append(split.strip()[:-3])
                elif '[e]' not in split:
                    ListOfMetabolites.append(split.strip())
print(len(ListOfMetabolites))

1321


In [67]:
####Try to identify correct KEGG ID

met_replace_dict = {}
met_replace_dict['UDP-N-acetyl-D_quinovosamine'] = 'NC00001'
met_replace_dict['4,4-diapophytofluene'] = 'C19840'
met_replace_dict['4,4-diapo-z-carotene'] = 'C19841'
met_replace_dict['4,4-diaponeurosporene'] = 'C16145'
met_replace_dict['Dehydrosuqalene'] = 'C16144'
met_replace_dict['1,2-diacylglycerol-4-O?밾omoserine'] = 'NC00002'
met_replace_dict['1,2-diacylglycerol-4-O-homoserin'] = 'NC00002'
met_replace_dict['1,2-diacylglycerol-4-O-(?쁍??Methyl)homoserine'] = 'NC00003'
met_replace_dict['1,2-diacylglycerol-4-O-(N-Methyl)homoserine'] = 'NC00003'
met_replace_dict['aminoacyl-diacyglycerol-lipoprotein'] = 'NC00004'
met_replace_dict['aminoacyl-diacyglyceryl-lipoprotein'] = 'NC00005'
met_replace_dict['Prolipoprotein'] = 'NC00006'
met_replace_dict['Diacylglyceryl-prolipoprotein'] = 'NC00007'
met_replace_dict['HPR-serine'] = 'NC00008'
met_replace_dict['HPR-serine_phosphate'] = 'NC000022'
met_replace_dict['S-lipoate'] = 'C00725'
met_replace_dict['adenosine-tRNAx'] = 'NC00009'
met_replace_dict['inosine-tRNAx'] = 'NC00010'
met_replace_dict['UDP-N-acetyl-D-quinovosamine'] = 'NC00001'
met_replace_dict['UDP-N-acetyl-D-fucosamine'] = 'NC00011'
met_replace_dict['2-oxo-4-hexenedioate'] = 'NC00012'
met_replace_dict['2-oxo-3-hexenedioate'] = 'NC00013'
met_replace_dict['Isobutylaldehyde'] = 'cpd24683'
met_replace_dict['1-carboxymethylpyridinium'] = 'cpd25980'
met_replace_dict['2-dimethylsulfoniopropionate'] = 'NC00014'
met_replace_dict['3-dimethylsulfoniopropionate'] = 'NC00015'
met_replace_dict['p-aminobenzoylglutamate'] = 'cpd26527'
met_replace_dict['gamma-butyrobetaine'] = 'C01181'
met_replace_dict['homobetaine'] = 'NC00016'
met_replace_dict['hydroxyethylthiazole'] = 'NC00017'
met_replace_dict['macrolide'] = 'cpd29400'
met_replace_dict['MgHPO4'] = 'NC00018'
met_replace_dict['MnHPO4'] = 'NC00019'
met_replace_dict['MoO4-2'] = 'C06232'
met_replace_dict['phosphite'] = 'C06701'
met_replace_dict['phosphonates'] = 'C06701_2'
met_replace_dict['xanthine'] = 'C00385'
met_replace_dict['ZnHPO4'] = 'NC00020'
met_replace_dict['PO4'] = 'C00009'
met_replace_dict['Hydroxymethylpyrimidine'] = 'C01279'
met_replace_dict['D-mannosamine-6-phosphate'] = 'NC00021'
met_replace_dict['Antimonite'] = 'cpd11598'
met_replace_dict['1,2-diacylglycerol-4-O-(N,N-dimethyl)homoserine'] = 'NC00022'
met_replace_dict['1,2-diacylglycerol-4-O-(N,N,N-trimethyl)homoserine'] = 'NC00023'
met_replace_dict['sucrose'] = 'C00089'
met_replace_dict['CaHPO4'] = 'NC00024'
met_replace_dict['CoHPO4']= 'NC00025'
#met_replace_dict['Fe3'] = 'C14819'
met_replace_dict['Fe3-anguibactin'] = 'NC00026'
met_replace_dict['Fe3-dicitrate'] = 'C06229'
met_replace_dict['Fe3-ferrichrome'] = 'C06228'

In [70]:
ListOfMetabolites_adapted = []
for met in ListOfMetabolites:
    if met in met_replace_dict.keys():
        ListOfMetabolites_adapted.append(met_replace_dict[met])
    else:
        ListOfMetabolites_adapted.append(met)
print(len(ListOfMetabolites_adapted))

1321


In [71]:
ListOfCytosolMetabolites = []
ListOfExternalMetabolites = []

for reaction in df['Reaction_Ids'].tolist():
    splitting = reaction.split()
    for split in splitting:
        if '(?' in split or '(N' in split: 
            if '[e]' in split and split[:-3] not in ListOfMetabolites:
                ListOfExternalMetabolites.append(split.strip()[:-3])
            elif '[e]' not in split and split not in ListOfCytosolMetabolites:
                ListOfCytosolMetabolites.append(split.strip())
        elif '+' not in split and '>' not in split and '<' not in split and '(' not in split:
            if '[e]' in split and split not in ListOfExternalMetabolites:
                ListOfExternalMetabolites.append(split[:-3])
            elif '[e]' not in split and split not in ListOfCytosolMetabolites:
                ListOfCytosolMetabolites.append(split)
print(len(ListOfCytosolMetabolites))
print(len(ListOfExternalMetabolites))

1315
251


In [72]:
DictOfMetNames = {}
DictOfFormulas = {}

for metabolite in ListOfMetabolites_adapted:
    try:
        result = REST.kegg_get(metabolite).read()
        
        for line in result.split('\n'):
            if 'NAME' in line:
                DictOfMetNames[metabolite] = line[12:].split(';')[0]
            if 'FORMULA' in line:
                DictOfFormulas[metabolite] = line[12:]
    except:
        continue
print(len(DictOfMetNames))
print(len(DictOfFormulas))

1236
1162


## Build the model

In [10]:
####
#Initialize Model
####

model = Model('Lee_Pan_Model')

In [11]:
####
#Initialize Metabolites
####

for met in ListOfCytosolMetabolites:
    new_metabolite = Metabolite(met+'_c',name = met, compartment = 'c')
    model.add_metabolites([new_metabolite])
    
for met in ListOfExternalMetabolites:
    new_metabolite = Metabolite(met+'_e', name = met, compartment = 'e')
    model.add_metabolites([new_metabolite])
    
print(len(model.metabolites))

1435


In [12]:
####
#Add Names and charges, where available
# Change ID where necessary 
###
doubled_metabolites= ['1,2-diacylglycerol-4-O-homoserin_c', 'UDP-N-acetyl-D-quinovosamine_c', 
                      'gamma-butyrobetaine_c', 'xanthine_c', 'PO4_c', 'Hydroxymethylpyrimidine_c', 
                     'PO4_e', '1,2-diacylglycerol-4-O-(N-Methyl)homoserine_c', 
                      '1,2-diacylglycerol-4-O-(N,N-dimethyl)homoserine', 'sucrose', 'Fe3']


for met in model.metabolites:
    if met.id[:-2] in DictOfMetNames:
        met.name = DictOfMetNames[met.id[:-2]]
    else:
        met.name = met.id[:-2]

#for met in model.metabolites:
#    if met.id[:-2] in DictOfFormulas:
#        met.formula = DictOfFormulas[met.id[:-2]]

for met in model.metabolites:
    if met.id in doubled_metabolites:
        continue
    elif met.id[:-2] in met_replace_dict.keys():
        met.id = met_replace_dict[met.id[:-2]]+ met.id[-2:]
        #print(met.id)
        
###ToDo: Delete doubled_metabolites!

In [13]:
#####
#Reactions
#####
direction_indicators = ['<=>', '-->', '<--']

for i in range(len(df)):
    if df.loc[i]['KEGG reaction ID'] not in model.reactions:
        reaction = Reaction(df.loc[i]['KEGG reaction ID'])

        #Upper and lower bounds
        direction = [split for split in df.loc[i]['Reaction_Ids'].split() if split in direction_indicators]
        if direction[0] == '<=>':
            reaction.upper_bound = 1000
            reaction.lower_bound = -1000
        elif direction[0] == '-->':
            reaction.upper_bound = 1000
            reaction.lower_bound = 0
        elif direction[0] == '<--':
            reaction.upper_bound = 0
            reaction.lower_bound = -1000

        #Reaction equation
        reaction_string = df.loc[i]['Reaction_Ids']
        reaction_string = reaction_string.replace('-->', '<=>')
        reaction_string = reaction_string.replace('<--', '<=>')

        substrates = reaction_string.split('<=>')[0].split('+')
        products = reaction_string.split('<=>')[1].split('+')

        for substrate in substrates:
            if '[e]' in substrate:
                if substrate.strip()[:-3] in met_replace_dict.keys():
                    reaction.add_metabolites({model.metabolites.get_by_id(met_replace_dict[substrate.strip()[:-3]]+'_e'): -1})
                elif '(' in substrate:
                    number = substrate.split()[0].strip()[1:-1]
                    reaction.add_metabolites({model.metabolites.get_by_id(substrate.strip()[:-3].split()[1]+'_e'): -int(number)})
                else:
                    reaction.add_metabolites({model.metabolites.get_by_id(substrate.strip()[:-3]+'_e'): -1})
            else:
                if substrate.strip() in met_replace_dict.keys():
                    reaction.add_metabolites({model.metabolites.get_by_id(met_replace_dict[substrate.strip()]+'_c'): -1})
                elif '(' in substrate:
                    #print(substrate)
                    number = substrate.split()[0].strip()[1:-1]
                    reaction.add_metabolites({model.metabolites.get_by_id(substrate.split()[1].strip()+'_c'): -int(number)})
                else:
                    reaction.add_metabolites({model.metabolites.get_by_id(substrate.strip()+'_c'): -1})


        for product in products:
            if '[e]' in product:
                if product.strip()[:-3] in met_replace_dict.keys():
                    reaction.add_metabolites({model.metabolites.get_by_id(met_replace_dict[product.strip()[:-3]]+'_e'): 1})
                elif '(' in product:
                    #print(product)
                    number = product.split()[0].strip()[1:-1]
                    reaction.add_metabolites({model.metabolites.get_by_id(product.split()[1].strip()[:-3]+'_e'): int(number)})
                else:
                    reaction.add_metabolites({model.metabolites.get_by_id(product.strip()[:-3]+'_e'): -1})
            else:
                if product.strip() in met_replace_dict.keys():
                    reaction.add_metabolites({model.metabolites.get_by_id(met_replace_dict[product.strip()]+'_c'): 1})
                elif '(' in product:
                    #print(product)
                    number = product.split()[0].strip()[1:-1]
                    reaction.add_metabolites({model.metabolites.get_by_id(product.split()[1].strip()+'_c'): int(number)})
                else:
                    reaction.add_metabolites({model.metabolites.get_by_id(product.strip()+'_c'): 1})

        model.add_reactions([reaction])
        
print(len(model.reactions))

1373


In [14]:
changed_react_names = 0
for react in model.reactions:
    if react.id in react_name_dict.keys():
        react.name = react_name_dict[react.id]
        changed_react_names += 1
print(changed_react_names)

1297


In [15]:
###Remove reactions that are empty
model.remove_reactions(['R04168', 'R00124'])

In [16]:
for met in model.metabolites:
    if (len(met.reactions)) == 0:
        print(met.id)

C00718_c
C00653_c
G10558_c
C00461_c
G10483_c
G10506_c
C00965_c
G10477_c
G10481_c
C03541_c
1,2-diacylglycerol-4-O-(N-Methyl)homoserine_c
1,2-diacylglycerol-4-O-homoserin_c
UDP-N-acetyl-D-quinovosamine_c
gamma-butyrobetaine_c
xanthine_c
PO4_c
Hydroxymethylpyrimidine_c
PO4_e


In [17]:
for met in model.metabolites:
    if (len(met.reactions)) == 0:
        if met.id[0] != 'C' and met.id[0] != 'G':
            model.remove_metabolites([met])

In [18]:
for met in model.metabolites:
    if (len(met.reactions)) == 0:
        print(met.id)

C00718_c
C00653_c
G10558_c
C00461_c
G10483_c
G10506_c
C00965_c
G10477_c
G10481_c
C03541_c
Hydroxymethylpyrimidine_c


In [19]:
cobra.io.write_sbml_model(model, '../Models/Lee_Pan_model/Initial_Lee.xml')

### Continue with libSBML

In [11]:
from libsbml import *

reader = SBMLReader()
document = reader.readSBML('../Models/Lee_Pan_model/Initial_Lee.xml')
print('Errors: ', document.getNumErrors())
lib_model = document.getModel()

Errors:  0


In [21]:
#add the missing metabolites
def add_react_prod(species):
    species = lib_model.getSpecies(species)
    react.addReactant(species)
    react.addProduct(species)


species_list = lib_model.getListOfSpecies()

for react in lib_model.getListOfReactions():
    if react.getId() == 'R_R07261':
        add_react_prod('M_C00718_c')
    elif react.getId() == 'R_R04083':
        add_react_prod('M_C00653')
    elif react.getId() == 'R_R06235':
        add_react_prod('M_G10558')
    elif react.getId() == 'R_01206':
        add_react_prod('M_C00461')
    elif react.getId() == 'R_R02334':
        add_react_prod('M_C00461')
    elif react.getId() == 'R_R06082':
        add_react_prod('M_G10483')
    elif react.getId() == 'R_R06239':
        add_react_prod('M_G10506')
    elif react.getId() == 'R_R00308':
        add_react_prod('M_C00965')
    elif react.getId() == 'R_R06204':
        add_react_prod('M_G10477')
    elif react.getId() == 'R_R06200':
        add_react_prod('M_G10481')
    elif react.getId() == 'R_R04241':
        add_react_prod('M_C03541')

In [22]:
### Add E.C. numbers to model

EC_df = {}
for i in range(len(df)):
    if df.loc[i]['KEGG reaction ID'] in EC_df.keys():
        EC_df[df.loc[i]['KEGG reaction ID']].append(df.loc[i]['EC number'])
    else:
        EC_df[df.loc[i]['KEGG reaction ID']] = [df.loc[i]['EC number']]
#print(len(EC_df))


for react in lib_model.getListOfReactions():
    if not react.isSetMetaId():
        react.setMetaId(react.getId())
    
        
    for i in range(len(EC_df[react.getId()[2:]])):
        cv = CVTerm()
        cv.setQualifierType(BIOLOGICAL_QUALIFIER)
        cv.setBiologicalQualifierType(BQB_IS)
        cv.addResource("https://identifiers.org/ec-code:" + EC_df[react.getId()[2:]][i])

        react.addCVTerm(cv)

In [23]:
writeSBMLToFile(document, '../Models/Lee/Initial_Lee_libsbml.xml')
#cobra.io.validate_sbml_model('Initial_Lee_libsbml.xml')

1

## Biomass Reaction

In [24]:
def find_id(met_name):
    for met in model.metabolites:
        if met.name == met_name:
            print(met.id)

In [167]:
find_id('10-Formyltetrahydrofolate')

C00234_c


In [117]:
model = cobra.io.read_sbml_model('../Models/Lee_Pan_model/Initial_Lee.xml')

No objective coefficients in model. Unclear what should be optimized


### Heinemann Biomass Reaction

In [118]:
growth_heine = pd.read_csv('BIomass_Heinemann.csv', sep=';')
growth_heine.head()

,Biomass components (m_i),ID,Coefficient(c_i)
0,L-Alanine,C00041_c,"0,213054"
1,L-Arginine,C00062_c,"0,111232"
2,L-Asparagine,C00152_c,"0,1732"
3,L-Aspartate,C00441_c,"0,261826"
4,L-Cysteine,C00097_c,"0,0192"


In [119]:
reaction = Reaction('Biomass_Heinemann')
reaction.name = 'Biomass reaction based on Heinemann'
reaction.lower_bound = 0
reaction.upper_bound = 1000

biomass = Metabolite('Biomass', name='Biomass', compartment='c')

for i in range(len(growth_heine)):
    number = float(growth_heine.loc[i]['Coefficient(c_i)'].replace(',', '.'))
    reaction.add_metabolites({
        model.metabolites.get_by_id(growth_heine.loc[i]['ID']): -number 
    })
reaction.add_metabolites({biomass: 1})   

model.add_reaction(reaction) 

model.objective = 'Biomass_Heinemann'

In [120]:
print(len(model.reactions))

1372


### Becker Biomass Reaction

In [121]:
growth_becker = pd.read_csv('Biomass_Becker.csv', sep=';')
growth_becker.head()

,Biomass components (m_i),ID,Coefficient(c_i)
0,H2O,C00001_c,20000
1,ATP,C00002_c,20000
2,L-Glutamate,C00025_c,"9,68"
3,L-Glutamine,C00064_c,"8,72"
4,L-Alanine,C00041_c,"6,6"


In [122]:
reaction = Reaction('Biomass_Becker')
reaction.name = 'Biomass reaction based on Becker and Palsson'
reaction.lower_bound = 0
reaction.upper_bound = 1000

#biomass = Metabolite('Biomass', name='Biomass', compartment='c')

for i in range(len(growth_becker)):
    number = float(growth_becker.loc[i]['Coefficient(c_i)'].replace(',', '.'))
    reaction.add_metabolites({
        model.metabolites.get_by_id(growth_becker.loc[i]['ID']): -number 
    })
reaction.add_metabolites({biomass: 1})   

model.add_reaction(reaction) 

model.objective = 'Biomass_Becker'

In [170]:
model.optimize()

,fluxes,reduced_costs
R02767,0.0,0.0
R04533,0.0,-0.0
R04534,0.0,-0.0
R04536,0.0,-0.0
R04543,0.0,-0.0
...,...,...
EX_C00073_e,0.0,0.0
EX_C00152_e,0.0,0.0
EX_C00378_e,0.0,0.0
EX_C00155_e,0.0,0.0


In [123]:
cobra.io.write_sbml_model(model, '../Models/Lee_Pan_model/Initial_Lee_2a.xml')

## Exchange reactions

In [124]:
model = cobra.io.read_sbml_model('../Models/Lee_Pan_model/Initial_Lee_2a.xml')

In [125]:
for met in model.metabolites:
    if met.compartment == 'e':
        reaction = Reaction('EX_'+met.id)
        reaction.name = met.name + ' exchange'
        reaction.lower_bound = -1000
        reaction.upper_bound = 1000
        reaction.add_metabolites({
            met: -1
        })
        model.add_reaction(reaction)
print(len(model.reactions))

1492


In [5]:
model.optimize()

,fluxes,reduced_costs
R02767,0.0,0.0
R04533,0.0,0.0
R04534,0.0,-0.0
R04536,0.0,0.0
R04543,0.0,-0.0
...,...,...
EX_C00003_e,0.0,0.0
EX_C00073_e,-0.0,-0.0
EX_C00152_e,0.0,0.0
EX_C00378_e,0.0,0.0


In [38]:
for i in range(len(growth_heine)):
    print(growth_heine.loc[i]['ID'], growth_heine.loc[i]['Biomass components (m_i)'])
    reactions = list(model.metabolites.get_by_id(growth_heine.loc[i]['ID']).reactions)
    transport_react = [react.id for react in reactions if 'T' in react.id]
    for react in transport_react:
        print(model.reactions.get_by_id(react))
    print('------')

C00041_c L-Alanine
T00061: C00041_e + C00080_e --> C00041_c + C00080_c
T00062: C00041_e + C01330_e --> C00041_c + C01330_c
------
C00062_c L-Arginine
T00064: C00062_e + C00077_c + C00077_e --> C00062_c
T00011: C00062_e + C00080_e --> C00062_c + C00080_c
T00063: C00001_e + C00002_c + C00062_e --> C00008_c + C00009_c + C00062_c
------
C00152_c L-Asparagine
T00156: C00152_e --> C00152_c
------
C00441_c L-Aspartate
------
C00097_c L-Cysteine
T00127: C00097_c + C00097_e --> 
T00155: C00097_e --> C00097_c
------
C00064_c L-Glutamine
------
C00025_c L-Glutamate
T00068: C00025_e + C01330_e --> C00025_c + C01330_c
T00067: C00025_e + C00080_e --> C00025_c + C00080_c
------
C00037_c Glycine
T00136: C00037_e + C01330_e --> C00037_c + C01330_c
T00054: C00037_e + C00080_e --> C00037_c + C00080_c
------
C00135_c L-Histidine
------
C00407_c L-Isoleucine
T00069: C00080_e + C00407_c + C00407_e --> C00080_c
T00070: C00080_e + C00407_e --> C00080_c + C00407_c
------
C00123_c L-Leucine
T00073: C00080_e + C

In [126]:
cobra.io.write_sbml_model(model, '../Models/Lee_Pan_model/Initial_Lee_2a.xml')
#cobra.io.validate_sbml_model('Initial_Lee_3.xml')

----
## Model Polisher used on Pan Model 

Check, whether 
- boundaries and
- objective value $\checkmark$

are still the same! 

In [127]:
old_model = cobra.io.read_sbml_model('../Models/Lee_Pan_model/Initial_Lee_2a.xml')
new_model = cobra.io.read_sbml_model('../Models/Lee_Pan_model/Initial_Lee_polished2.xml')

In [128]:
boundaries_old = {}

for react in old_model.reactions:
    boundaries_old[react.id] = [reaction.lower_bound, reaction.upper_bound]
print(len(boundaries_old))

for react in new_model.reactions:
    if react in old_model.reactions:
        react.lower_bound = boundaries_old[react.id][0]
        react.upper_bound = boundaries_old[react.id][1]

1492


In [130]:
cobra.io.write_sbml_model(new_model, '../Models/Lee_Pan_model/Initial_Lee_polished2.xml')

In [131]:
new_model.reactions.get_by_id('BIOMASS_Becker')

Reaction identifier,BIOMASS_Becker
Name,
Memory address,0x0106c9bddd0
Stoichiometry,20000.0 C00001_c + 20000.0 C00002_c + 0.01 C00003_c + 0.01 C00006_c + 0.01 C00016_c + 3.11 C00020_c + 9.68 C00025_c + 3.51 C00037_c + 6.6 C00041_c + 5.45 C00047_c + 2.2 C00055_c + 3.88 C00062_c + 8... 20000.0 H2O + 20000.0 ATP + 0.01 NAD+ + 0.01 NADP+ + 0.01 FAD + 3.11 AMP + 9.68 L-Glutamate + 3.51 Glycine + 6.6 L-Alanine + 5.45 L-Lysine + 2.2 CMP + 3.88 L-Arginine + 8.72 L-Glutamine + 2.6 L-Ser...
GPR,
Lower bound,0.0
Upper bound,1000.0


-----
# Create individual models

In [132]:
model = cobra.io.read_sbml_model('../Models/Lee_Pan_model/Initial_Lee_polished2.xml')
print(len(model.reactions))
print(len(model.metabolites))

1492
1429


In [17]:
df.head()

,EC number,KEGG reaction ID,Reaction_Ids,Reaction_metNames,Newman,TCH1516,USA300,Mu3,JH9,JH1,COL,RF122,Mu50,MW2,MSSA476,MRSA252,N315
0,1.1.1.100,R02767,C01271 + C00006 <=> C00685 + C00005 + C00080,(3R)-3-Hydroxyacyl-[acyl-carrier protein] + NA...,+,+,+,+,+,+,+,+,+,+,+,+,+
1,1.1.1.100,R04533,C04618 + C00006 <=> C05744 + C00005,(3R)-3-Hydroxybutanoyl-[acyl-carrier protein] ...,+,+,+,+,+,+,+,+,+,+,+,+,+
2,1.1.1.100,R04534,C04619 + C00006 <=> C05753 + C00005,(3R)-3-Hydroxydecanoyl-[acyl-carrier protein] ...,+,+,+,+,+,+,+,+,+,+,+,+,+
3,1.1.1.100,R04536,C04620 + C00006 <=> C05750 + C00005,(3R)-3-Hydroxyoctanoyl-[acyl-carrier protein] ...,+,+,+,+,+,+,+,+,+,+,+,+,+
4,1.1.1.100,R04543,C04633 + C00006 <=> C05762 + C00005,(3R)-3-Hydroxypalmitoyl-[acyl-carrier protein]...,+,+,+,+,+,+,+,+,+,+,+,+,+


In [108]:
individual_biomass = pd.read_excel('supp_191_12_4015__SupplementaryDataset1.xls', sheet_name='biomass Becker and Palsson ',
                       skiprows=2)
individual_biomass.head()

,Biomass components (m_i),Coefficient(c_i),Newman,TCH1516,USA300,Mu3,JH9,JH1,COL,RF122,Mu50,MW2,MSSA476,MRSA252,N315
0,H2O,20000.00,+,+,+,+,+,+,+,+,+,+,+,+,+
1,ATP,20000.00,+,+,+,+,+,+,+,+,+,+,+,+,+
2,L-Glutamate,9.68,+,+,+,+,+,+,+,+,+,+,+,+,+
3,L-Glutamine,8.72,+,+,+,+,+,+,+,+,+,+,+,+,+
4,L-Alanine,6.60,+,+,+,+,+,+,+,+,+,+,+,+,+


In [133]:
def create_individual_models(strain):
    model = cobra.io.read_sbml_model('../Models/Lee_Pan_model/Initial_Lee_polished2.xml')
    
    #reactions
    remove_react = [df.loc[i]['KEGG reaction ID'] for i in range(len(df)) if df.loc[i][strain] != "+"]
    model.remove_reactions(remove_react)
    
    #metabolites
    remove_met = [met for met in model.metabolites if len(met.reactions) == 0]
    model.remove_metabolites(remove_met)
    
    #Biomass
    biomass_react = model.reactions.get_by_id('BIOMASS_Becker')

    remove_biomass_comp = [individual_biomass.loc[i]['Biomass components (m_i)'] for i in range(len(individual_biomass)) if individual_biomass.loc[i][strain] != "+"]
    remove_comp = [[growth_becker.loc[growth_becker['Biomass components (m_i)'] == comp]['ID']][0].iloc[0] for comp in remove_biomass_comp]
    remove_biomass_coeff = [individual_biomass.loc[i]['Coefficient(c_i)'] for i in range(len(individual_biomass)) if individual_biomass.loc[i][strain] != "+"]
    #print(remove_comp)

    for metabolite, coefficient in zip(remove_comp, remove_biomass_coeff):
        biomass_react.subtract_metabolites({
            model.metabolites.get_by_id(metabolite): -coefficient
        })
    return(model)

### Step 1

In [134]:
##Create individual models

for strain in df.keys()[4:]:
    model = create_individual_models(strain)
    model.id = strain
    model.name = 'Staphylococcus aureus ' + strain
    cobra.io.write_sbml_model(model, '../Models/Lee/Lee_' + strain + '.xml')

----
# Additional information

In [3]:
#Import Biopython modules to interact with KEGG
#https://widdowquinn.github.io/2018-03-06-ibioic/02-sequence_databases/09-KEGG_programming.html

from Bio import SeqIO
from Bio.KEGG import REST
#from Bio.KEGG.KGML import KGML_parser
#from Bio.Graphics.KGML_vis import KGMLCanvas

import csv

import os

In [75]:
model = cobra.io.read_sbml_model('../Models/Lee_Pan_model/Initial_Lee_polished2.xml')
metabolite_dict = {}
metabolites_not_in_kegg = []

for met in model.metabolites:
    try:
        result = REST.kegg_get('cpd:' + met.id[:-2]).read()
        result = result.split('\n')
        for elem in result:
            if 'FORMULA' in elem:
                formula = elem.split()[1].strip()
                metabolite_dict[met.id[:-2]] = formula
                #print(metabolite_dict[met.id[:-2]], formula)
                #w = csv.writer(open("Metabolites.csv", "w"))
                #for key, val in metabolite_dict.items():
                #    w.writerow([key, val])
    except:
        metabolites_not_in_kegg.append(met.id)
print(len(metabolites_not_in_kegg))

118


### Step 2

In [135]:
### Add chemical formulas to all models:
directory = '../Models/Lee/'

for filename in os.listdir(directory):
    if filename.endswith(".xml") : 
        model = cobra.io.read_sbml_model(os.path.join(directory, filename))
        print(model.id)
        for met in model.metabolites:
            if met.id[:-2] in metabolite_dict:
                met.formula = metabolite_dict[met.id[:-2]]
        cobra.io.write_sbml_model(model, os.path.join(directory, filename))
    else:
        continue

Mu3
N315
Mu50
JH9
MSSA476
TCH1516
RF122
JH1
MRSA252
Newman
COL
MW2
USA300
JH1


In [8]:
###First get List of Enzymes (use Pan model for that): 
enzyme_dict = {}
reactions_not_in_kegg = []

model = cobra.io.read_sbml_model('../Models/Lee_Pan_model/Initial_Lee_polished.xml')

for reaction in model.reactions:
    try: 
        result = REST.kegg_get('rn:' + reaction.id).read()
        result = result.split('\n')
        for elem in result:
            if 'ENZYME' in elem:
                enzymes = elem.split()[1:]
                enzyme_dict[reaction] = enzymes

                #with open("Reaction_Enzyme.csv", 'w') as file:
                #    writer = csv.writer(file)
                #    for key, val in enzyme_dict.items():
                #        writer.writerow([key, val])
    except:
        reactions_not_in_kegg.append(reaction.id)
print(len(reactions_not_in_kegg))

347


In [77]:
strain_dictionary = {'Newman': 'SAE', 'Mu3': 'SAW', 'N315': 'SAU', 'Mu50': 'SAV', 
                    'JH9': 'SAJ', 'MSSA476': 'SAS', 'TCH1516': 'SAX', 'MRSA252': 'SAR', 
                    'COL': 'SAC', 'MW2': 'SAM', 'USA300': 'SAA', 'JH1': 'SAH'}

In [185]:
'''#create list of unique enzymes: 
enzyme_list = []
strain_dictionary = {'Newman': 'SAE', 'Mu3': 'SAW', 'N315': 'SAU', 'Mu50': 'SAV', 
                    'JH9': 'SAJ', 'MSSA476': 'SAS', 'TCH1516': 'SAX', 'MRSA252': 'SAR', 
                    'COL': 'SAC', 'MW2': 'SAM', 'USA300': 'SAA', 'JH1': 'SAH'}


for reaction in enzyme_dict.keys():
    enzyme_list += enzyme_dict[reaction]
    
print(len(enzyme_list))
enzyme_list = list(set(enzyme_list))
print(len(enzyme_list))

gene_df = pd.DataFrame(columns = ['E.C', 'Strain', 'Genes'])


enzymes = ['1.1.1.100', '1.1.1.-']
for enzyme in enzyme_list:
    try:
        result = REST.kegg_get('ec:' + enzyme).read()
        result = result.split('\n')
        for elem in result:
            for value in strain_dictionary.values():
                if '            '+value+': ' in elem:
                    #print(enzyme, value, elem.split()[1:])
                    gene_df = gene_df.append({'E.C': enzyme, 'Strain': value, 'Genes': elem.split()[1:]}, ignore_index = True)
                    gene_df.to_csv('Genes.csv')
    except:
        print(enzyme)
print(gene_df)'''

1529
817
1.14.13.-
1.1.1.-
AND
1.7.99.-
GLUCOSE
RELATED
1.2.1.-
3.1.2.-
CATALYZED
1.5.1.-
1.14.-.-
1.3.1.-
SPECIFICITY
2.7.8.-
ENZYMATICALLY
ANOMERIC
ANOMERIZATION
2.7.1.-
1.3.99.-
SPONTANEOUS
ENZYMES.
3.1.3.-
1.14.11.-
6-PHOSPHATE
2.5.1.-
3.4.-.-
2.3.1.-
2.6.1.-
2.4.2.-
OF
1.3.8.7
3.6.1.-
4.1.2.-
4.2.1.-
            E.C Strain                                         Genes
0       1.2.4.1    SAU                [SA0943-1(pdhA), SA0944(pdhB)]
1       1.2.4.1    SAV                [SAV1093(pdhA), SAV1094(phdB)]
2       1.2.4.1    SAW            [SAHV_1085(pdhA), SAHV_1086(phdB)]
3       1.2.4.1    SAH                  [SaurJH1_1175, SaurJH1_1176]
4       1.2.4.1    SAJ                  [SaurJH9_1153, SaurJH9_1154]
5       1.2.4.1    SAM                        [MW0976, MW0977(phdB)]
6       1.2.4.1    SAS                            [SAS1028, SAS1029]
7       1.2.4.1    SAR                [SAR1067(pdhA), SAR1068(pdhB)]
8       1.2.4.1    SAC            [SACOL1102(pdhA), SACOL1103(pdhB)]
9  

In [78]:
gene_df = pd.read_csv('Genes.csv')
gene_df = gene_df.drop(['Unnamed: 0'], axis=1)
for i in range(len(gene_df)):
    gene_df.loc[i]['Genes'] = gene_df.loc[i]['Genes'].strip('][').split(', ') # Converting string to list
    #print(type(gene_df.loc[i]['Genes']))
gene_df.head()


,E.C,Strain,Genes
0,1.2.4.1,SAU,"['SA0943-1(pdhA)', 'SA0944(pdhB)']"
1,1.2.4.1,SAV,"['SAV1093(pdhA)', 'SAV1094(phdB)']"
2,1.2.4.1,SAW,"['SAHV_1085(pdhA)', 'SAHV_1086(phdB)']"
3,1.2.4.1,SAH,"['SaurJH1_1175', 'SaurJH1_1176']"
4,1.2.4.1,SAJ,"['SaurJH9_1153', 'SaurJH9_1154']"


## Add Enzymes and Genes to Models

In [79]:
from libsbml import *
writer = SBMLWriter()
reader = SBMLReader()

reactions_enzymes = pd.read_csv('Reaction_Enzyme.csv', header = None)
reactions_enzymes = reactions_enzymes.rename(columns={0: 'Reaction', 1: 'EC'})
for i in range(len(reactions_enzymes)):
    reactions_enzymes.loc[i]['Reaction'] = reactions_enzymes.loc[i]['Reaction'].split(':')[0].strip()


print(reactions_enzymes.head())

  Reaction                                                EC
0   R02767                          ['1.1.1.100', '1.1.1.-']
1   R04533  ['1.1.1.100', '1.1.1.-', '2.3.1.85', '2.3.1.86']
2   R04534  ['1.1.1.100', '1.1.1.-', '2.3.1.85', '2.3.1.86']
3   R04536  ['1.1.1.100', '1.1.1.-', '2.3.1.85', '2.3.1.86']
4   R04543  ['1.1.1.100', '1.1.1.-', '2.3.1.85', '2.3.1.86']


### Step 3

In [136]:
'''
Add EC Numbers as Annotations

'''
directory = '../Models/Lee/'

for filename in os.listdir(directory):
    if filename.endswith(".xml"): 
        document = reader.readSBML(os.path.join(directory, filename))
        lib_model = document.getModel()
        
        for reaction in lib_model.getListOfReactions():
            if reaction.getId()[2:] not in reactions_not_in_kegg:
                if reaction.getId()[2:] in reactions_enzymes['Reaction'].tolist():
                    enzymes = reactions_enzymes.loc[reactions_enzymes['Reaction'] == reaction.getId()[2:]]['EC'].iloc[0]
                    enzymes = enzymes.strip('][').split(', ') 
                    enzymes = [e.replace("'", '') for e in enzymes]

                    for enzyme in enzymes:
                        cv = CVTerm()
                        cv.setQualifierType(BIOLOGICAL_QUALIFIER)
                        cv.setBiologicalQualifierType(BQB_IS)
                        cv.addResource('https://identifiers.org/ec-code/'+enzyme)
                        reaction.addCVTerm(cv)
        
        writer.writeSBML(document, os.path.join(directory, filename))
        
    else:
        continue

In [137]:
'''Add Genes to the respective Models'''
directory = '../Models/Lee/'

for filename in os.listdir(directory):
    if filename.endswith(".xml"): 
        model = cobra.io.read_sbml_model(os.path.join(directory, filename))
        if model.id != 'RF122':
            strain = strain_dictionary[model.id]
            print(model.id)
            
            temp = gene_df.loc[gene_df['Strain'] == strain]

            for react in model.reactions:
                #GPR = ''
                try:
                    enzymes = reactions_enzymes.loc[reactions_enzymes['Reaction'] == react.id]['EC'].iloc[0]
                    enzymes = enzymes.strip('][').split(', ') 
                    enzymes = [e.replace("'", '') for e in enzymes]
                    #print(react.id, enzymes)
                    GPR = ''

                    for enzyme in enzymes:
                        try:
                            genes = temp.loc[temp['E.C'] == enzyme]['Genes'].iloc[0]

                            for gene in genes:
                                if '(' in gene:
                                    GPR += gene.split('(')[0].replace("'", '') + ' or '
                                else:
                                    GPR += gene.replace("'", '') + ' or '

                            react.gene_reaction_rule = GPR[:-4]
                        except:
                            continue

                except:
                    continue
                
            cobra.io.write_sbml_model(model, os.path.join(directory, filename))


Mu3
N315
Mu50
JH9
MSSA476
TCH1516
JH1
MRSA252
Newman
COL
MW2
USA300
JH1


In [138]:
directory = '../Models/Lee/'

for filename in os.listdir(directory):
    if filename.endswith(".xml"): 
        model = cobra.io.read_sbml_model(os.path.join(directory, filename))
        print(model.id)
        print(len(model.reactions))
        print(len(model.metabolites))
        print(len(model.genes))
        print('------------------')


Mu3
1483
1412
492
------------------
N315
1492
1418
493
------------------
Mu50
1486
1413
492
------------------
JH9
1488
1417
506
------------------
MSSA476
1477
1411
488
------------------
TCH1516
1434
1375
476
------------------
RF122
1466
1388
0
------------------
JH1
1479
1409
500
------------------
MRSA252
1481
1405
488
------------------
Newman
1463
1403
481
------------------
COL
1477
1407
486
------------------
MW2
1481
1409
492
------------------
USA300
1485
1411
506
------------------
JH1
1479
1409
500
------------------


### Annotations+

In [144]:
for filename in os.listdir(directory):
    if filename.endswith(".xml"): 

        document= reader.readSBML(os.path.join(directory, filename))
        model = document.getModel()

        for react in model.getListOfReactions():
            if react.id[2:] not in reactions_not_in_kegg:
                cv = CVTerm()
                cv.setQualifierType(BIOLOGICAL_QUALIFIER)
                cv.setBiologicalQualifierType(BQB_IS)
                cv.addResource("https://identifiers.org/kegg.reaction/" + react.id[2:])

                react.addCVTerm(cv)


        for species in model.getListOfSpecies():
            species.setMetaId(species.getId())
            if species.id[2:] not in metabolites_not_in_kegg:
                cv = CVTerm()
                cv.setQualifierType(BIOLOGICAL_QUALIFIER)
                cv.setBiologicalQualifierType(BQB_IS)
                cv.addResource("https://identifiers.org/kegg.compound/" + species.id[2:-2])

                species.addCVTerm(cv)

        for species in model.getListOfSpecies():
            splugin = species.getPlugin('fbc')
            if splugin is not None:
                if '(' in splugin.getChemicalFormula() or 'n' in splugin.getChemicalFormula() or '.' in splugin.getChemicalFormula():
                    formula_old = splugin.getChemicalFormula()
                    formula = formula_old.replace('(', '')
                    formula = formula.replace(')', '')
                    formula = formula.replace('n', '')
                    formula = formula.replace('.', '')
                    formula = formula.replace('-2', '')
                    splugin.setChemicalFormula(formula)

                    note = "<body xmlns='http://www.w3.org/1999/xhtml'><p>FORMULA: " + formula_old + '</p></body>'
                    species.setNotes(note)

            species.setSBOTerm(247)

        mplugin = model.getPlugin('fbc') 

        for gene in mplugin.getListOfGeneProducts():
            gene.setMetaId(gene.getId())
            gene.setSBOTerm(243)
            cv = CVTerm()
            cv.setQualifierType(BIOLOGICAL_QUALIFIER)
            cv.setBiologicalQualifierType(BQB_IS)
            cv.addResource("https://identifiers.org/kegg.genes/" + gene.id[2:])
            gene.addCVTerm(cv)

        writeSBMLToFile(document, os.path.join(directory, filename))

In [54]:
cobra.io.validate_sbml_model('../Models/Lee/Lee_JH1_annotated.xml')

(<Model JH1 at 0x103661dd50>,
 {'SBML_FATAL': [],
  'SBML_ERROR': [],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': [],
  'COBRA_FATAL': [],
  'COBRA_ERROR': [],
  'COBRA_WARNING': [],
  'COBRA_CHECK': []})

In [11]:
directory = '../Models/Lee/'

reactions = []
metabolites = []
genes = []

for filename in os.listdir(directory):
    if filename.endswith(".xml"): 
        model = cobra.io.read_sbml_model(os.path.join(directory, filename))
        print(model.optimize().objective_value)
        reactions.append(len(model.reactions))
        metabolites.append(len(model.metabolites))
        genes.append(len(model.genes))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [9]:
print(int(np.mean(reactions)), int(np.std(reactions)))
print(int(np.mean(metabolites)), int(np.std(metabolites)))
#print(int(np.mean(genes.remove(0))), int(np.std(genes.remove(0))))
print(int(np.mean(genes)), int(np.std(genes)))

1476 14
1406 11
491 8


In [10]:
print(genes)

[492, 493, 492, 506, 488, 476, 488, 481, 486, 492, 506, 500]
